In [3]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
# from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.options import Options
import re
import datetime
import time

%matplotlib inline

In [42]:
str(datetime.datetime.now())

'2019-02-08 19:24:06.563393'

In [29]:
with open('test1','w') as f:
        f.write('test')

In [33]:
with open('test1','a') as f:
        f.write("oranges"+"\n")

In [34]:
with open('test1','r') as f:
        a = f.read()

In [36]:
print(a)

testoranges
oranges



In [7]:
directory = pd.read_json('topLevel.json').sort_index()

In [3]:
directory.head()

,country,lang,more,page,url
0,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
1,阿富汗,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
2,东亚峰会（EAS）,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gjhd...
3,韩国,zh,NaN,重要文件,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
4,蒙古,zh,NaN,外交掠影,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...


In [4]:
directory.shape

(1536, 5)

In [5]:
directory.loc[directory['country'] == '阿富汗',]

,country,lang,more,page,url
0,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
1,阿富汗,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
697,阿富汗,zh,NaN,外交掠影,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
773,阿富汗,zh,NaN,重要文件,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
950,阿富汗,zh,NaN,重要讲话,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
1397,阿富汗,zh,NaN,发言人有关谈话,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...


In [6]:
# options = Options()
# options.add_argument("--headless")
# driver = webdriver.Firefox(firefox_options=options, executable_path="geckodriver.exe")
# print("Firefox Headless Browser Invoked")

In [7]:
options = Options()
options.add_argument("--headless")
options.add_argument('log-level=1')     
driver = webdriver.Chrome(options=options, executable_path="chromedriver.exe")
print("Chrome Headless Browser Invoked")

Chrome Headless Browser Invoked


In [8]:
documents = pd.DataFrame(columns=list(directory.columns) + ['nav','title','date'])

In [9]:
documents.head()

,country,lang,more,page,url,nav,title,date


In [10]:
try:
    documents_orig = pd.read_json('documents.json')
except:
    documents_orig = pd.DataFrame(columns=list(directory.columns) + ['nav','title','date'])

In [12]:
documents_orig.head()

,country,lang,more,page,url,nav,title,date
0,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部副部长孔铉佑会见阿富汗离任驻华大使莫萨扎伊(2018-09-06),2018-09-06
1,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,习近平会见阿富汗总统加尼(2018-06-10),2018-06-10
10,尼泊尔,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_尼泊尔_驻外报道_,第八届中国高等教育展在加德满都揭幕(2013-05-14),2013-05-14
100,尼泊尔,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_尼泊尔_驻外报道_,驻尼泊尔大使杨厚兰接受新华社采访(2013-01-15),2013-01-15
1000,哥斯达黎加,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_北美洲_哥斯达黎加_驻外报道_,驻哥斯达黎加大使汤恒拜会哥新任第一副总统兼外长坎贝尔(2018-05-17),2018-05-17


In [13]:
for i in directory.index:
    url_root = directory.loc[i,'url']
    print(url_root)
    country = directory.loc[i,'country']
    page = directory.loc[i,'page']
    lang = directory.loc[i,'lang']

    driver.get(url_root)
    soup=BeautifulSoup(driver.page_source, 'lxml')

    nav = str()
    for k in soup.find(class_='nav').find_all('a'):
        nav = nav + k.text + '_'

    try:
        more = soup.find("a", string='尾页')['href']
        print(more)
        directory.loc[i,'more'] = more
    except:
        print('NO MORE')
    
    for j in soup.find(class_='rebox_news').find_all('li'):
        title = j.text
        url_new = url_root+j.find('a')['href'][2:]

        try:
            date = re.findall(r"\d\d\d\d\-\d\d\-\d\d",j.text)[0]
        except:
            date = '9999-99-99'
        print(date)
    
        documents = documents.append({'country':country,
                                      'page':page,
                                      'url':url_new,
                                      'lang':lang,
                                      'nav':nav,
                                      'title':title,
                                      'date':date,
                                     },
                                     ignore_index=True)
#     except:
#         print(directory.loc[i,'url'])
#         break

https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676207/xgxw_676213/
default_16.shtml
2019-01-25
2019-01-11
2018-12-29
2018-12-15
2018-12-15
2018-12-15
2018-12-15
2018-12-15
2018-11-30
2018-10-13
2018-09-06
2018-06-10
2018-05-30
2018-05-30
2018-05-29
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676207/1206x2_676229/
default_52.shtml
2019-01-31
2019-01-30
2019-01-30
2019-01-29
2019-01-28
2019-01-25
2019-01-24
2019-01-22
2019-01-21
2019-01-20
2019-01-18
2019-01-17
2019-01-17
2019-01-16
2019-01-16
https://www.fmprc.gov.cn/web/gjhdq_676201/gjhdqzz_681964/dyfheas_682566/xgxw_682572/
default_3.shtml
2018-11-15
2018-08-04
2017-11-15
2017-08-08
2017-08-07
2016-09-09
2016-07-26
2016-06-29
2015-11-23
2015-11-23
2015-11-22
2015-11-22
2015-11-22
2015-11-20
2015-08-06
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676524/1207_676536/
NO MORE
2018-05-10
2018-05-09
2015-11-02
2014-07-04
2014-07-04
2013-06-27
2011-01-25
2010-05-30
2000-11-07


NO MORE
2015-05-23
2000-11-07
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679498/ywfc_679520/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676332/ywfc_676354/
NO MORE
2014-04-22
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679522/xgxw_679528/
NO MORE
2018-09-07
2015-08-25
2015-01-16
2014-08-20
2014-08-17
2013-10-22
2007-04-25
2007-04-25
2007-04-24
2006-02-07
2002-12-04
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679522/1206x2_679542/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679522/1207_679534/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679522/1209_679532/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679522/fyrygth_679530/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gjhdqzz_681964/lhg_683190/fyrth_683198/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679522/ywfc

NO MORE
2000-11-07
2000-11-07
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676620/1207_676632/
NO MORE
2018-07-09
2009-05-12
2004-07-07
2000-11-07
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/nmz_680924/1206_681120/1209_681130/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/1206_677970/xgxw_677976/
default_1.shtml
2018-09-06
2018-09-06
2018-03-19
2016-10-06
2015-10-09
2013-07-18
2013-07-08
2013-06-06
2009-09-19
2009-06-04
2009-06-04
2007-10-20
2005-12-02
2005-12-02
2005-12-01
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/nmz_680924/1206_681120/fyrygth_681128/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/nmz_680924/1206_681120/ywfc_681142/
NO MORE
2012-06-30
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/nmz_680924/1206_681144/xgxw_681150/
default_1.shtml
2018-12-18
2018-09-24
2018-09-24
2018-09-24
2018-09-24
2017-12-07
2017-11-23
2017-09-15
2017-08-07
2016-05-29
2013-06-26
2013-06-26
2013-06-21
2013-06-03
201

NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/nmz_680924/1206_681216/1209_681226/
NO MORE
2018-01-22
2016-11-23
2011-06-11
2006-09-07
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676548/fyrygth_676556/
default_1.shtml
2016-08-30
2010-12-23
2010-06-29
2010-06-28
2010-06-15
2010-06-15
2010-04-16
2010-04-07
2009-02-05
2006-11-09
2006-11-07
2006-11-07
2006-08-03
2005-03-29
2005-03-28
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/nmz_680924/1206_681216/fyrygth_681224/
NO MORE
2018-08-13
2015-05-12
2015-05-11
2015-03-31
2010-02-28
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676380/1206x2_676400/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gjhdqzz_681964/lhg_682782/zywj_682794/
NO MORE
2014-01-17
2011-05-04
2004-07-07
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679642/1207_679654/
NO MORE
2016-11-06
2016-06-19
2013-08-26
2009-08-20
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/nsd_678

NO MORE
2002-07-04
2002-07-04
2002-07-03
2002-04-16
2002-04-16
2002-04-16
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/1206_677656/ywfc_677678/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681418/1206x2_681438/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681418/1207_681430/
NO MORE
2000-11-07
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676788/1207_676800/
NO MORE
2018-12-18
2017-12-16
2016-08-20
2016-03-23
2016-03-23
2015-11-12
2015-09-04
2014-11-15
2013-04-06
2003-07-01
2000-11-07
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681418/1209_681428/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681418/fyrygth_681426/
NO MORE
2008-05-17
2003-11-13
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681418/ywfc_681440/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gjhdqzz_681964/yzxhhy_683118/fyrth_683126/
NO MORE
2002

...

NO MORE
2002-04-19
2002-04-19
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_678916/1209_678926/
NO MORE
2018-07-06
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_678916/fyrygth_678924/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_678916/ywfc_678938/
NO MORE
2012-09-10
2012-07-31
2011-05-27
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/1206_677824/1209_677834/
NO MORE
2015-11-30
https://www.fmprc.gov.cn/web/gjhdq_676201/gjhdqzz_681964/lhg_682998/xgxw_683004/
NO MORE
2011-06-04
2010-04-29
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679618/fyrygth_679626/
NO MORE
2015-06-05
2015-03-30
2015-03-27
2015-03-23
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/bmz_679954/1206_680008/xgxw_680014/
NO MORE
2018-03-17
2015-09-14
2014-08-27
2014-08-26
2013-06-03
2013-01-01
2012-03-16
2011-01-02
2011-01-01
2010-12-20
2008-01-01
2007-09-10
2006-09-26
2004-11-18
2004-06-15
https://www.fmprc.

NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679012/ywfc_679034/
default_1.shtml
2012-12-02
2012-09-30
2012-09-27
2012-04-06
2012-04-06
2012-04-06
2012-04-06
2012-04-06
2012-04-06
2012-04-06
2012-02-03
2011-12-22
2011-04-20
2010-10-08
2010-09-26
https://www.fmprc.gov.cn/web/gjhdq_676201/gjhdqzz_681964/shlshl_682710/zyjh_682720/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/bmz_679954/1206_680528/1207_680540/
default_1.shtml
2017-09-27
2016-09-04
2016-06-09
2016-04-01
2016-04-01
2015-09-26
2015-09-26
2015-07-01
2015-06-25
2015-06-25
2014-11-12
2014-07-11
2014-07-09
2014-02-15
2013-07-13
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679062/xgxw_679068/
default_6.shtml
2018-09-27
2018-06-22
2018-06-21
2018-06-20
2018-02-23
2018-02-07
2017-11-02
2017-10-30
2017-09-27
2017-05-04
2017-05-03
2017-03-13
2016-12-06
2016-04-18
2016-04-17
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679062/1206x2_679082/
defaul

default_39.shtml
2019-01-27
2019-01-27
2019-01-23
2019-01-10
2019-01-09
2019-01-06
2018-12-31
2018-12-21
2018-12-20
2018-12-19
2018-12-19
2018-12-13
2018-12-13
2018-12-11
2018-12-11
https://www.fmprc.gov.cn/web/gjhdq_676201/gjhdqzz_681964/lhg_683022/xgxw_683028/
default_8.shtml
2019-01-07
2019-01-04
2019-01-04
2018-12-13
2018-09-05
2018-09-05
2018-08-08
2018-07-01
2018-05-29
2018-05-10
2018-03-19
2018-02-08
2018-02-08
2018-02-08
2018-02-08
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676207/fyrygth_676215/
default_3.shtml
2016-07-24
2015-08-14
2015-07-30
2015-06-23
2015-05-29
2015-05-25
2015-05-04
2015-04-24
2015-03-13
2014-11-24
2014-07-18
2014-07-15
2014-04-05
2014-04-03
2009-12-03
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676476/1209_676486/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679738/fyrygth_679746/
NO MORE
2002-07-07
2002-07-07
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679

default_14.shtml
2019-01-21
2019-01-18
2019-01-17
2019-01-17
2019-01-04
2018-12-28
2018-12-14
2018-12-09
2018-12-05
2018-11-30
2018-11-28
2018-11-24
2018-11-21
2018-11-19
2018-11-14
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/1206_677608/fyrygth_677616/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/bmz_679954/1206_680826/xgxw_680832/
default_1.shtml
2018-06-20
2018-05-15
2018-05-14
2018-01-23
2016-07-07
2016-06-22
2014-06-20
2014-06-20
2014-02-27
2014-02-25
2013-06-07
2013-06-03
2013-06-03
2013-06-03
2013-06-02
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679258/1207_679270/
NO MORE
2006-07-13
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679258/1209_679268/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676476/ywfc_676498/
NO MORE
2012-04-07
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679258/fyrygth_679266/
NO MORE
2006-06-01
2003-02-08
https://www.fmprc.gov.cn

default_3.shtml
2018-09-01
2018-09-01
2018-06-12
2018-03-20
2018-03-06
2017-07-12
2017-06-22
2016-08-16
2016-04-19
2016-04-19
2016-04-18
2015-12-04
2015-07-17
2015-06-10
2015-05-28
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679354/1207_679366/
NO MORE
2016-11-06
2000-11-07
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/nmz_680924/1206_681096/ywfc_681118/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679354/1209_679364/
NO MORE
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676788/1206x2_676808/
default_66.shtml
2018-12-28
2018-12-03
2018-11-27
2018-11-27
2018-11-19
2018-11-12
2018-10-29
2018-10-26
2018-10-18
2018-10-02
2018-09-28
2018-09-27
2018-09-25
2018-09-14
2018-09-11
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679354/fyrygth_679362/
NO MORE
2015-06-10
2015-02-10
https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/oz_678770/1206_679906/1207_679918/
NO MORE
2018-09-25
2016-12-21
201

In [14]:
documents

,country,lang,more,page,url,nav,title,date
0,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部部长助理陈晓东会见阿富汗高级和平委员会秘书长达乌德扎伊(2019-01-25),2019-01-25
1,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,王毅会见阿富汗总统国家安全顾问莫希卜(2019-01-11),2019-01-11
2,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部阿富汗事务特使邓锡军访问巴基斯坦(2018-12-29),2018-12-29
3,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,阿富汗总统加尼会见王毅(2018-12-15),2018-12-15
4,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,王毅会见阿富汗首席执行官阿卜杜拉(2018-12-15),2018-12-15
5,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,王毅谈中阿巴外长对话达成六点共识(2018-12-15),2018-12-15
6,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,王毅：推动阿富汗和解进程早日迈出新步伐，获得新突破(2018-12-15),2018-12-15
7,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,中国-阿富汗-巴基斯坦三方外长举行第二次对话(2018-12-15),2018-12-15
8,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部阿富汗事务特使邓锡军出席阿富汗问题日内瓦部长级会议(2018-11-30),2018-11-30
9,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,李克强会见阿富汗首席执行官阿卜杜拉(2018-10-13),2018-10-13


In [15]:
documents = pd.concat([documents,documents_orig]).drop_duplicates(subset=['url','country']).reset_index(drop=True)

In [16]:
documents.to_json('documents.json')

In [17]:
documents.to_csv('documents.csv')

In [18]:
documents.shape

(128648, 8)

In [19]:
for i in soup.find(class_='rebox_news').find_all('li'):
    print(i.text)

In [20]:
s = 'cats pajamas'

In [21]:
pd.read_json('documents.json')

,country,lang,more,page,url,nav,title,date
0,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部部长助理陈晓东会见阿富汗高级和平委员会秘书长达乌德扎伊(2019-01-25),2019-01-25
1,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,王毅会见阿富汗总统国家安全顾问莫希卜(2019-01-11),2019-01-11
10,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部副部长孔铉佑会见阿富汗离任驻华大使莫萨扎伊(2018-09-06),2018-09-06
100,罗马尼亚,zh,NaN,外交掠影,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_欧洲_罗马尼亚_外交掠影_,同中国的对话(2009-05-06),2009-05-06
1000,澳大利亚,zh,NaN,重要文件,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_大洋洲_澳大利亚_重要文件_,二十国集团领导人发表布里斯班峰会公报(2014-11-17),2014-11-17
10000,柬埔寨,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_柬埔寨_驻外报道_,驻柬埔寨大使布建国拜会柬副首相兼内政大臣韶肯(2013-08-07),2013-08-07
100000,玻利维亚,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_南美洲_玻利维亚_驻外报道_,驻玻利维亚大使吴元山出席向玻最高法院提供捐助协议签字仪式(2016-10-11),2016-10-11
100001,玻利维亚,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_南美洲_玻利维亚_驻外报道_,驻玻利维亚大使吴元山会见玻发规部长奥莱利亚纳(2016-10-05),2016-10-05
100002,玻利维亚,zh,NaN,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_南美洲_玻利维亚_驻外报道_,驻玻利维亚大使吴元山会见玻最高法院院长玛玛尼(2016-10-02),2016-10-02
100003,日本,zh,NaN,发言人有关谈话,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_日本_发言人有关谈话_,外交部发言人秦刚就日本政府公布中学历史教科书审定结果发表谈话(2005-04-05),2005-04-05
